# Introduction

This notebook serves as a comprehensive guide to the core algorithms implemented for this project, with a specific focus on unit testing. Each section presents a self-contained algorithm, including its purpose, detailed code implementation, and a clear demonstration of its functionality.

The primary goal of this document is to ensure the technical integrity of the codebase. By isolating and testing each function, we can verify that the data generation, preference modeling, voting rules (Plurality, Veto, and Borda), and distortion calculations are all working as intended. This process is crucial for establishing a reliable foundation before moving on to larger-scale integration tests and the final experimental runs.

# Import Libralies

In [1]:
import numpy as np
import random
import collections

## Utility Generation with Distributions

- This script provides a function to generate a set of cardinal utility values for agents and alternatives based on different probability distributions.

- Utility values represent the "true numerical value" an agent has for an alternative. These values are used to calculate the optimal social welfare, which is needed to
determine the distortion of a voting rule.

In [2]:
def generate_utilities(num_agents, num_alternatives, distribution_type='uniform'):
    """
    Generates a 2D array of cardinal utility values based on a specified distribution.

    Args:
        num_agents (int): The number of agents (voters).
        num_alternatives (int): The number of alternatives.
        distribution_type (str): The type of distribution to use.
                                 'uniform': Utilities are randomly and uniformly distributed between 0 and 1.
                                 'normal': Utilities follow a normal (Gaussian) distribution.
                                 'strong_preference': One alternative has a utility of 1.00, others are 0.00.
                                 'unit_sum': All utilities for a single agent sum to 1.
    Returns:
        list of list: A nested list where utilities[i][j] is the utility of alternative j for agent i.
    """
    utilities = []
    
    # Check for valid distribution type
    valid_distributions = ['uniform', 'normal', 'strong_preference', 'unit_sum']
    if distribution_type not in valid_distributions:
        print(f"Warning: Invalid distribution type '{distribution_type}'. Defaulting to 'uniform'.")
        distribution_type = 'uniform'
        
    # Generate utilities based on the specified distribution
    for _ in range(num_agents):
        agent_utilities = []
        if distribution_type == 'uniform':
            # Uniform distribution: each utility value is equally likely to be between 0 and 1.
            agent_utilities = [random.uniform(0, 1) for _ in range(num_alternatives)]
        elif distribution_type == 'normal':
            # Normal distribution: values are clustered around a mean.
            agent_utilities = [np.random.normal(loc=0.5, scale=0.2) for _ in range(num_alternatives)]
            # We'll clip the values to ensure they stay within the [0, 1] range.
            agent_utilities = [max(0, min(1, u)) for u in agent_utilities]
        elif distribution_type == 'strong_preference':
            # Strong Preference: one alternative is a clear favorite (1.00), all others are worthless (0.00).
            agent_utilities = [0.00] * num_alternatives
            strong_choice_index = random.randint(0, num_alternatives - 1)
            agent_utilities[strong_choice_index] = 1.00
        elif distribution_type == 'unit_sum':
            # Unit-Sum: utilities are randomly distributed such that their sum is 1.
            # We generate random points and use them to partition the interval [0, 1].
            points = sorted([0.0] + [random.uniform(0, 1) for _ in range(num_alternatives - 1)] + [1.0])
            agent_utilities = [points[i+1] - points[i] for i in range(num_alternatives)]
            
        utilities.append(agent_utilities)
        
    return utilities


In [21]:
if __name__ == "__main__":
    # --- Example 1: strong_preference Distribution ---
    num_agents_uniform = 10
    num_alternatives_uniform = 5
    uniform_utilities = generate_utilities(num_agents_uniform, num_alternatives_uniform, 'strong_preference')

    print("=== strong_preference Distribution Utilities ===")
    print(f"Number of Agents: {num_agents_uniform}")
    print(f"Number of Alternatives: {num_alternatives_uniform}")
    print("Generated Utilities:")
    for i, u_list in enumerate(uniform_utilities):
        print(f"Agent {i+1}: {[round(u, 2) for u in u_list]}")

=== strong_preference Distribution Utilities ===
Number of Agents: 10
Number of Alternatives: 5
Generated Utilities:
Agent 1: [0.0, 0.0, 0.0, 1.0, 0.0]
Agent 2: [0.0, 0.0, 1.0, 0.0, 0.0]
Agent 3: [1.0, 0.0, 0.0, 0.0, 0.0]
Agent 4: [0.0, 0.0, 0.0, 0.0, 1.0]
Agent 5: [0.0, 1.0, 0.0, 0.0, 0.0]
Agent 6: [0.0, 0.0, 1.0, 0.0, 0.0]
Agent 7: [0.0, 0.0, 1.0, 0.0, 0.0]
Agent 8: [0.0, 0.0, 0.0, 0.0, 1.0]
Agent 9: [1.0, 0.0, 0.0, 0.0, 0.0]
Agent 10: [0.0, 0.0, 0.0, 0.0, 1.0]


In [4]:
if __name__ == "__main__":
    # --- Example 1: unit_sum Distribution ---
    num_agents_uniform = 10
    num_alternatives_uniform = 2
    uniform_utilities = generate_utilities(num_agents_uniform, num_alternatives_uniform, 'unit_sum')

    print("=== unit_sum Distribution Utilities ===")
    print(f"Number of Agents: {num_agents_uniform}")
    print(f"Number of Alternatives: {num_alternatives_uniform}")
    print("Generated Utilities:")
    for i, u_list in enumerate(uniform_utilities):
        print(f"Agent {i+1}: {[round(u, 2) for u in u_list]}")

=== unit_sum Distribution Utilities ===
Number of Agents: 10
Number of Alternatives: 2
Generated Utilities:
Agent 1: [0.16, 0.84]
Agent 2: [0.03, 0.97]
Agent 3: [0.61, 0.39]
Agent 4: [0.95, 0.05]
Agent 5: [0.13, 0.87]
Agent 6: [0.35, 0.65]
Agent 7: [0.71, 0.29]
Agent 8: [0.73, 0.27]
Agent 9: [0.98, 0.02]
Agent 10: [0.71, 0.29]


In [5]:
if __name__ == "__main__":
    # --- Example 1: Uniform Distribution ---
    num_agents_uniform = 10
    num_alternatives_uniform = 5
    uniform_utilities = generate_utilities(num_agents_uniform, num_alternatives_uniform, 'uniform')

    print("=== Uniform Distribution Utilities ===")
    print(f"Number of Agents: {num_agents_uniform}")
    print(f"Number of Alternatives: {num_alternatives_uniform}")
    print("Generated Utilities:")
    for i, u_list in enumerate(uniform_utilities):
        print(f"Agent {i+1}: {[round(u, 2) for u in u_list]}")

=== Uniform Distribution Utilities ===
Number of Agents: 10
Number of Alternatives: 5
Generated Utilities:
Agent 1: [0.1, 0.31, 0.88, 0.7, 0.86]
Agent 2: [0.39, 0.39, 0.87, 0.06, 1.0]
Agent 3: [0.13, 0.32, 0.11, 0.79, 0.46]
Agent 4: [0.37, 0.54, 0.92, 0.8, 0.11]
Agent 5: [0.21, 0.14, 0.41, 0.44, 0.88]
Agent 6: [0.08, 0.45, 0.13, 0.95, 0.87]
Agent 7: [0.55, 0.63, 0.52, 0.97, 0.11]
Agent 8: [0.7, 0.06, 0.96, 0.19, 0.24]
Agent 9: [0.94, 0.88, 0.13, 0.76, 0.06]
Agent 10: [0.61, 0.53, 0.99, 0.68, 0.66]


In [6]:
if __name__ == "__main__":    
# --- Example 2: Normal Distribution ---
    print("\n" + "="*40 + "\n")
    num_agents_normal = 10
    num_alternatives_normal = 5
    normal_utilities = generate_utilities(num_agents_normal, num_alternatives_normal, 'normal')

    print("=== Normal Distribution Utilities ===")
    print(f"Number of Agents: {num_agents_normal}")
    print(f"Number of Alternatives: {num_alternatives_normal}")
    print("Generated Utilities:")
    for i, u_list in enumerate(normal_utilities):
        print(f"Agent {i+1}: {[round(u, 2) for u in u_list]}")



=== Normal Distribution Utilities ===
Number of Agents: 10
Number of Alternatives: 5
Generated Utilities:
Agent 1: [0.6, 0.69, 0.42, 0.32, 0.67]
Agent 2: [0.75, 0.59, 0.26, 0.34, 0.18]
Agent 3: [0.16, 0.79, 0.36, 0.77, 0.62]
Agent 4: [0.56, 0.52, 0.93, 0.6, 0.28]
Agent 5: [0.56, 0.24, 0.66, 0.38, 0.48]
Agent 6: [0.43, 0.43, 0.64, 0.63, 0.02]
Agent 7: [0.37, 0.61, 0.29, 0.31, 0.68]
Agent 8: [0.7, 0.13, 0.08, 0.81, 0.09]
Agent 9: [0.66, 0.56, 0.72, 0.47, 0.61]
Agent 10: [0.46, 0.4, 0.54, 0.56, 0.63]


# Algorithm Implementation

## Plurality Voting Algorithm Implementation

- This script demonstrates the Plurality voting rule. It first generates a set of random
preferences for a given number of agents and alternatives. Then, it runs the Plurality
algorithm to find the winning alternative, which is the one with the most first-place votes.

In [7]:
def generate_random_preference_list(num_alternatives):
    """
    Generates a random permutation of alternatives to represent an agent's preferences.

    Args:
        num_alternatives (int): The total number of alternatives.

    Returns:
        list: A randomly ordered list of alternative names (e.g., ['A3', 'A1', 'A2']).
    """
    alternatives = [f'A{i + 1}' for i in range(num_alternatives)]
    random.shuffle(alternatives)
    return alternatives

def generate_preferences(num_agents, num_alternatives):
    """
    Simulates preferences for a given number of agents and alternatives.
    Each agent's preference is a random permutation of the alternatives.

    Args:
        num_agents (int): The number of agents (voters).
        num_alternatives (int): The number of alternatives.

    Returns:
        list: A list of lists, where each inner list is an agent's preference list.
    """
    preferences = []
    for _ in range(num_agents):
        preferences.append(generate_random_preference_list(num_alternatives))
    return preferences

def plurality_voting(preferences):
    """
    Implements the Plurality voting rule to find the winning alternative.
    Plurality works by giving one point to the first-ranked alternative of each agent.
    The alternative with the most points wins.

    Args:
        preferences (list): A 2D list of agent preferences.

    Returns:
        dict: A dictionary containing the winner, their vote count, and all scores.
    """
    if not preferences:
        return {"winner": None, "votes": 0, "scores": {}}

    # Tally the votes. Each agent's top-ranked choice gets one point.
    vote_counts = collections.Counter(pref[0] for pref in preferences)

    # Find the winner by identifying the alternative with the most votes
    winner = None
    max_votes = -1

    for alternative, votes in vote_counts.items():
        if votes > max_votes:
            max_votes = votes
            winner = alternative
        # Tie-breaking can be added here if needed, but for simplicity, the first found winner is returned.

    return {
        "winner": winner,
        "votes": max_votes,
        "scores": dict(vote_counts)
    }


In [8]:
# Run it
if __name__ == "__main__":
    num_agents = 10
    num_alternatives = 5

    # Generate preferences and run the Plurality algorithm
    agent_preferences = generate_preferences(num_agents, num_alternatives)
    results = plurality_voting(agent_preferences)

    # Print the results
    print("=== Plurality Voting Simulation ===")
    print(f"Number of Agents: {num_agents}")
    print(f"Number of Alternatives: {num_alternatives}")

    print("\nSimulated Preferences:")
    for i, pref in enumerate(agent_preferences):
        print(f"Agent {i + 1}: {' > '.join(pref)}")

    print("\nVoting Results:")
    print("Scores:", results['scores'])
    print(f"Winner: {results['winner']} with {results['votes']} votes.")

=== Plurality Voting Simulation ===
Number of Agents: 10
Number of Alternatives: 5

Simulated Preferences:
Agent 1: A2 > A5 > A4 > A3 > A1
Agent 2: A1 > A4 > A3 > A2 > A5
Agent 3: A3 > A1 > A4 > A5 > A2
Agent 4: A5 > A3 > A1 > A2 > A4
Agent 5: A4 > A1 > A2 > A5 > A3
Agent 6: A3 > A4 > A5 > A1 > A2
Agent 7: A2 > A3 > A5 > A1 > A4
Agent 8: A4 > A5 > A1 > A3 > A2
Agent 9: A5 > A4 > A1 > A2 > A3
Agent 10: A2 > A4 > A3 > A5 > A1

Voting Results:
Scores: {'A2': 3, 'A1': 1, 'A3': 2, 'A5': 2, 'A4': 2}
Winner: A2 with 3 votes.


## Veto Voting Algorithm Implementation

This script implements the Veto voting rule. It takes a set of agent preferences and
determines the winning alternative. 

The Veto rule works by giving one negative point
(a 'veto') to the last-ranked alternative of each agent. The alternative with the fewest
vetoes (the highest score) wins.

In [9]:
def generate_random_preference_list(num_alternatives):
    """
    Generates a random permutation of alternatives to represent an agent's preferences.

    Args:
        num_alternatives (int): The total number of alternatives.

    Returns:
        list: A randomly ordered list of alternative names (e.g., ['A3', 'A1', 'A2']).
    """
    alternatives = [f'A{i + 1}' for i in range(num_alternatives)]
    random.shuffle(alternatives)
    return alternatives

def generate_preferences(num_agents, num_alternatives):
    """
    Simulates preferences for a given number of agents and alternatives.
    Each agent's preference is a random permutation of the alternatives.

    Args:
        num_agents (int): The number of agents (voters).
        num_alternatives (int): The number of alternatives.

    Returns:
        list: A list of lists, where each inner list is an agent's preference list.
    """
    preferences = []
    for _ in range(num_agents):
        preferences.append(generate_random_preference_list(num_alternatives))
    return preferences

def veto_voting(preferences):
    """
    Implements the Veto voting rule to find the winning alternative.
    Veto works by giving one negative point to the last-ranked alternative of each agent.
    The alternative with the highest total score (fewest vetoes) wins.

    Args:
        preferences (list): A 2D list of agent preferences.

    Returns:
        dict: A dictionary containing the winner, their score, and all scores.
    """
    if not preferences or not preferences[0]:
        return {"winner": None, "score": 0, "scores": {}}

    num_alternatives = len(preferences[0])
    # The default score for all alternatives is zero.
    veto_scores = {alt: 0 for alt in preferences[0]}

    # Iterate through each agent's preferences.
    for preference_list in preferences:
        # The last-ranked alternative gets a negative point (or a veto).
        last_choice = preference_list[num_alternatives - 1]
        veto_scores[last_choice] += 1  # Increment the veto count

    # Find the winner: the alternative with the fewest vetoes.
    winner = None
    min_vetoes = float('inf')  # Start with a very high number to find the minimum.

    # Iterate through the scores to find the alternative with the minimum vetoes.
    for alternative, score in veto_scores.items():
        # The Veto rule is a bit counter-intuitive: the lowest score wins.
        if score < min_vetoes:
            min_vetoes = score
            winner = alternative
        # Tie-breaking can be added here if needed, but for simplicity, the first one is returned.
    
    # We can also represent the score positively for clarity (e.g., as 10 - number of vetoes)
    # This might make the output more readable.
    scores_for_display = {alt: len(preferences) - score for alt, score in veto_scores.items()}

    return {
        "winner": winner,
        "score": len(preferences) - min_vetoes,
        "scores": scores_for_display
    }

## Run it with a sample preferences list

In [10]:
# Run it
if __name__ == "__main__":
    # You can reuse the generate_preferences function from your previous code.
    # For this example, let's use a sample preference list to demonstrate the logic.
    sample_preferences = [
        ['A1', 'A2', 'A3', 'A4'],  # A4 gets a veto
        ['A2', 'A3', 'A4', 'A1'],  # A1 gets a veto
        ['A4', 'A3', 'A2', 'A1'],  # A1 gets a veto
        ['A3', 'A1', 'A2', 'A4'],  # A4 gets a veto
        ['A2', 'A4', 'A1', 'A3'],  # A3 gets a veto
    ]
    
    print("=== Veto Voting Simulation ===")
    print("Simulated Preferences:")
    for i, pref in enumerate(sample_preferences):
        print(f"Agent {i + 1}: {' > '.join(pref)}")
    
    veto_results = veto_voting(sample_preferences)
    
    print("\nVeto Scores:")
    print(veto_results['scores'])
    print(f"Winner: {veto_results['winner']} with a score of {veto_results['score']}.")

=== Veto Voting Simulation ===
Simulated Preferences:
Agent 1: A1 > A2 > A3 > A4
Agent 2: A2 > A3 > A4 > A1
Agent 3: A4 > A3 > A2 > A1
Agent 4: A3 > A1 > A2 > A4
Agent 5: A2 > A4 > A1 > A3

Veto Scores:
{'A1': 3, 'A2': 5, 'A3': 4, 'A4': 3}
Winner: A2 with a score of 5.


## Run it with generate_preferences def

In [11]:
# Run it
if __name__ == "__main__":
    # You can reuse the generate_preferences function from your previous code.
    num_agents = 10
    num_alternatives = 5
    agent_preferences = generate_preferences(num_agents, num_alternatives)
    
    print("=== Veto Voting Simulation ===")
    print(f"Number of Agents: {num_agents}")
    print(f"Number of Alternatives: {num_alternatives}")
    print("\nSimulated Preferences:")
    for i, pref in enumerate(agent_preferences):
        print(f"Agent {i + 1}: {' > '.join(pref)}")
    
    veto_results = veto_voting(agent_preferences)
    
    print("\nVeto Scores:")
    print(veto_results['scores'])
    print(f"Winner: {veto_results['winner']} with a score of {veto_results['score']}.")


=== Veto Voting Simulation ===
Number of Agents: 10
Number of Alternatives: 5

Simulated Preferences:
Agent 1: A5 > A2 > A1 > A4 > A3
Agent 2: A2 > A4 > A5 > A1 > A3
Agent 3: A3 > A5 > A4 > A1 > A2
Agent 4: A4 > A1 > A5 > A2 > A3
Agent 5: A1 > A2 > A4 > A5 > A3
Agent 6: A5 > A3 > A2 > A4 > A1
Agent 7: A5 > A3 > A4 > A1 > A2
Agent 8: A1 > A3 > A2 > A4 > A5
Agent 9: A3 > A5 > A4 > A1 > A2
Agent 10: A2 > A4 > A5 > A1 > A3

Veto Scores:
{'A5': 9, 'A2': 7, 'A1': 9, 'A4': 10, 'A3': 5}
Winner: A4 with a score of 10.


## Borda Voting Algorithm Implementation

This script implements the Borda voting rule. It takes a set of agent preferences and determines the winning alternative. 

The Borda rule works by assigning points to each alternative based on its rank in an agent's preference list.

In [12]:
def generate_random_preference_list(num_alternatives):
    """
    Generates a random permutation of alternatives to represent an agent's preferences.

    Args:
        num_alternatives (int): The total number of alternatives.

    Returns:
        list: A randomly ordered list of alternative names (e.g., ['A3', 'A1', 'A2']).
    """
    alternatives = [f'A{i + 1}' for i in range(num_alternatives)]
    random.shuffle(alternatives)
    return alternatives

def generate_preferences(num_agents, num_alternatives):
    """
    Simulates preferences for a given number of agents and alternatives.
    Each agent's preference is a random permutation of the alternatives.

    Args:
        num_agents (int): The number of agents (voters).
        num_alternatives (int): The number of alternatives.

    Returns:
        list: A list of lists, where each inner list is an agent's preference list.
    """
    preferences = []
    for _ in range(num_agents):
        preferences.append(generate_random_preference_list(num_alternatives))
    return preferences

def borda_voting(preferences):
    """
    Implements the Borda voting rule to find the winning alternative.

    Args:
        preferences (list): A 2D list of agent preferences.

    Returns:
        dict: A dictionary containing the winner, their score, and all scores.
    """
    if not preferences or not preferences[0]:
        return {"winner": None, "score": 0, "scores": {}}
    
    num_alternatives = len(preferences[0])
    borda_scores = collections.defaultdict(int)

    # Calculate points for each alternative based on its rank in each preference list.
    for preference_list in preferences:
        # The first alternative gets num_alternatives - 1 points, the second gets num_alternatives - 2, etc.
        for i, alternative in enumerate(preference_list):
            points = num_alternatives - 1 - i
            borda_scores[alternative] += points
    
    # Find the winner: the alternative with the highest Borda score.
    winner = None
    max_score = -1

    for alternative, score in borda_scores.items():
        if score > max_score:
            max_score = score
            winner = alternative
    
    return {
        "winner": winner,
        "score": max_score,
        "scores": dict(borda_scores)
    }


In [13]:
# Run it
if __name__ == "__main__":
    num_agents = 10
    num_alternatives = 5
    agent_preferences = generate_preferences(num_agents, num_alternatives)
    
    print("=== Borda Voting Simulation ===")
    print(f"Number of Agents: {num_agents}")
    print(f"Number of Alternatives: {num_alternatives}")
    print("\nSimulated Preferences:")
    for i, pref in enumerate(agent_preferences):
        print(f"Agent {i + 1}: {' > '.join(pref)}")
    
    borda_results = borda_voting(agent_preferences)
    
    print("\nBorda Scores:")
    print(borda_results['scores'])
    print(f"Winner: {borda_results['winner']} with a score of {borda_results['score']}.")

=== Borda Voting Simulation ===
Number of Agents: 10
Number of Alternatives: 5

Simulated Preferences:
Agent 1: A1 > A4 > A5 > A3 > A2
Agent 2: A4 > A3 > A5 > A1 > A2
Agent 3: A3 > A4 > A5 > A2 > A1
Agent 4: A5 > A2 > A1 > A3 > A4
Agent 5: A5 > A4 > A2 > A1 > A3
Agent 6: A5 > A3 > A2 > A4 > A1
Agent 7: A1 > A2 > A5 > A4 > A3
Agent 8: A5 > A3 > A1 > A2 > A4
Agent 9: A5 > A1 > A3 > A4 > A2
Agent 10: A3 > A4 > A1 > A2 > A5

Borda Scores:
{'A1': 19, 'A4': 19, 'A5': 28, 'A3': 21, 'A2': 13}
Winner: A5 with a score of 28.


## Distortion Calculation Algorithm Implementation

This script provides a function to calculate distortion based on the cardinal utility values and the winning alternative from a voting rule.

In [14]:
def calculate_distortion(utilities, winner):
    """
    Calculates the distortion of a voting rule's outcome.

    The distortion is the ratio of the optimal social welfare (the best possible outcome)
    to the achieved social welfare (the outcome of the chosen alternative).

    Args:
        utilities (list): A 2D list of cardinal utility values for each agent and alternative.
                          utilities[i][j] is the utility of alternative j for agent i.
        winner (str): The name of the winning alternative as determined by a voting rule
                      (e.g., 'A3').

    Returns:
        dict: A dictionary containing the five key outputs for analysis.
    """
    if not utilities:
        return {
            "optimal_alternative": None,
            "optimal_social_welfare": 0,
            "chosen_alternative": winner,
            "achieved_social_welfare": 0,
            "distortion": 0
        }

    num_agents = len(utilities)
    num_alternatives = len(utilities[0])
    
    # Map alternative names (A1, A2, etc.) to their index (0, 1, etc.) for easier lookup
    alternative_names = [f'A{i + 1}' for i in range(num_alternatives)]
    alt_to_index = {name: i for i, name in enumerate(alternative_names)}
    
    # 1. Calculate the social welfare for all alternatives
    social_welfare = [0] * num_alternatives
    for j in range(num_alternatives):
        social_welfare[j] = sum(utilities[i][j] for i in range(num_agents))
        
    # 2. Find the optimal alternative and optimal social welfare
    optimal_social_welfare = max(social_welfare)
    optimal_index = social_welfare.index(optimal_social_welfare)
    optimal_alternative = alternative_names[optimal_index]
    
    # 3. Find the achieved social welfare of the winning alternative
    if winner in alt_to_index:
        winner_index = alt_to_index[winner]
        achieved_social_welfare = social_welfare[winner_index]
    else:
        achieved_social_welfare = 0 # If winner is not found, welfare is 0

    # 4. Calculate distortion
    # To avoid division by zero, check if achieved social welfare is positive.
    # Distortion is defined as Optimal Social Welfare / Achieved Social Welfare.
    if achieved_social_welfare > 0:
        distortion = optimal_social_welfare / achieved_social_welfare
    else:
        # If the achieved social welfare is zero, distortion is considered infinity
        # for practical purposes, but we can set it to a very large number for now.
        distortion = float('inf')

    return {
        "optimal_alternative": optimal_alternative,
        "optimal_social_welfare": optimal_social_welfare,
        "chosen_alternative": winner,
        "achieved_social_welfare": achieved_social_welfare,
        "distortion": distortion
    }

# === Example Usage ===
if __name__ == "__main__":
    # Example utility data (e.g., from your generate_utilities function)
    # These values represent the "true" worth of each alternative to each agent.
    sample_utilities = [
        [0.8, 0.2, 0.9, 0.4, 0.1], # Agent 1
        [0.3, 0.7, 0.5, 0.8, 0.6], # Agent 2
        [0.1, 0.9, 0.2, 0.5, 0.8], # Agent 3
    ]
    
    # Example winner from a voting rule (e.g., from your plurality_voting function)
    # Let's assume the Plurality winner was 'A4'
    plurality_winner = 'A4'

    # Calculate distortion for this scenario
    results = calculate_distortion(sample_utilities, plurality_winner)
    
    print("=== Distortion Calculation Results ===")
    print(f"Optimal Alternative: {results['optimal_alternative']}")
    print(f"Optimal Social Welfare: {results['optimal_social_welfare']:.2f}")
    print(f"Chosen Alternative (by rule): {results['chosen_alternative']}")
    print(f"Achieved Social Welfare: {results['achieved_social_welfare']:.2f}")
    print(f"Distortion: {results['distortion']:.2f}")

=== Distortion Calculation Results ===
Optimal Alternative: A2
Optimal Social Welfare: 1.80
Chosen Alternative (by rule): A4
Achieved Social Welfare: 1.70
Distortion: 1.06


---

# Combine The invloved funtions for each voting rules

The following blocks reveals self-contained code for each voting rule. They simulates three classic voting rules:

1. Plurality Voting: A simple "first-past-the-post" system where each agent votes for their most preferred alternative.

2. Veto Voting: An elimination-style rule where each agent "vetoes" their least preferred alternative. The alternative with the fewest vetoes wins.

2. Borda Voting: A ranked voting system where alternatives are assigned points based on their rank in each agent's preference list. The alternative with the highest total score wins.

For each simulation, the framework generates a set of agents with randomly assigned preferences and utilities. It then calculates the social welfare for every alternative—the sum of all agents' utilities for that option. This allows us to identify the optimal social welfare, which is the highest possible total utility, and the achieved social welfare of the alternative chosen by the voting rule.

# Self-contained Python script for the Plurality voting rule

This script provides a complete experimental framework for simulating the Plurality voting rule and calculating its distortion.

It combines all the previously developed functions into a single, cohesive programme that follows the UI/UX mock-up. It takes a single set of inputs (number of agents, number of alternatives, and distribution type) and outputs the key metrics for analysis.

In [15]:
# === PART 1: Data Generation Functions ===

def generate_utilities(num_agents, num_alternatives, distribution_type='uniform'):
    """
    Generates a 2D array of cardinal utility values based on a specified distribution.

    Args:
        num_agents (int): The number of agents (voters).
        num_alternatives (int): The number of alternatives.
        distribution_type (str): The type of distribution to use.
                                 'uniform': Utilities are randomly and uniformly distributed between 0 and 1.
                                 'normal': Utilities follow a normal (Gaussian) distribution.
                                 'strong_preference': One alternative has a utility of 1.00, others are 0.00.
                                 'unit_sum': All utilities for a single agent sum to 1.
    Returns:
        list of list: A nested list where utilities[i][j] is the utility of alternative j for agent i.
    """
    utilities = []
    
    # Check for valid distribution type
    valid_distributions = ['uniform', 'normal', 'strong_preference', 'unit_sum']
    if distribution_type not in valid_distributions:
        print(f"Warning: Invalid distribution type '{distribution_type}'. Defaulting to 'uniform'.")
        distribution_type = 'uniform'
        
    # Generate utilities based on the specified distribution
    for _ in range(num_agents):
        agent_utilities = []
        if distribution_type == 'uniform':
            # Uniform distribution: each utility value is equally likely to be between 0 and 1.
            agent_utilities = [random.uniform(0, 1) for _ in range(num_alternatives)]
        elif distribution_type == 'normal':
            # Normal distribution: values are clustered around a mean.
            agent_utilities = [np.random.normal(loc=0.5, scale=0.2) for _ in range(num_alternatives)]
            # We'll clip the values to ensure they stay within the [0, 1] range.
            agent_utilities = [max(0, min(1, u)) for u in agent_utilities]
        elif distribution_type == 'strong_preference':
            # Strong Preference: one alternative is a clear favorite (1.00), all others are worthless (0.00).
            agent_utilities = [0.00] * num_alternatives
            strong_choice_index = random.randint(0, num_alternatives - 1)
            agent_utilities[strong_choice_index] = 1.00
        elif distribution_type == 'unit_sum':
            # Unit-Sum: utilities are randomly distributed such that their sum is 1.
            # We generate random points and use them to partition the interval [0, 1].
            points = sorted([0.0] + [random.uniform(0, 1) for _ in range(num_alternatives - 1)] + [1.0])
            agent_utilities = [points[i+1] - points[i] for i in range(num_alternatives)]
            
        utilities.append(agent_utilities)
        
    return utilities

def generate_random_preference_list(num_alternatives):
    """
    Generates a random permutation of alternatives to represent an agent's preferences.
    """
    alternatives = [f'A{i + 1}' for i in range(num_alternatives)]
    random.shuffle(alternatives)
    return alternatives

def generate_preferences(num_agents, num_alternatives):
    """
    Simulates preferences for a given number of agents and alternatives.
    """
    preferences = []
    for _ in range(num_agents):
        preferences.append(generate_random_preference_list(num_alternatives))
    return preferences

# === PART 2: Voting Algorithm Function ===

def plurality_voting(preferences):
    """
    Implements the Plurality voting rule to find the winning alternative.
    """
    if not preferences:
        return {"winner": None, "votes": 0, "scores": {}}

    vote_counts = collections.Counter(pref[0] for pref in preferences)
    winner = None
    max_votes = -1

    for alternative, votes in vote_counts.items():
        if votes > max_votes:
            max_votes = votes
            winner = alternative

    return {
        "winner": winner,
        "votes": max_votes,
        "scores": dict(vote_counts)
    }

# === PART 3: Distortion Calculation Function ===

def calculate_distortion(utilities, winner):
    """
    Calculates the distortion of a voting rule's outcome.
    """
    if not utilities:
        return {
            "optimal_alternative": None,
            "optimal_social_welfare": 0,
            "chosen_alternative": winner,
            "achieved_social_welfare": 0,
            "distortion": 0
        }

    num_agents = len(utilities)
    num_alternatives = len(utilities[0])
    
    alternative_names = [f'A{i + 1}' for i in range(num_alternatives)]
    alt_to_index = {name: i for i, name in enumerate(alternative_names)}
    
    # 1. Calculate social welfare for all alternatives
    social_welfare = [0] * num_alternatives
    for j in range(num_alternatives):
        social_welfare[j] = sum(utilities[i][j] for i in range(num_agents))
        
    # 2. Find optimal alternative and optimal social welfare
    optimal_social_welfare = max(social_welfare)
    optimal_index = social_welfare.index(optimal_social_welfare)
    optimal_alternative = alternative_names[optimal_index]
    
    # 3. Find achieved social welfare of the winning alternative
    achieved_social_welfare = 0
    if winner in alt_to_index:
        winner_index = alt_to_index[winner]
        achieved_social_welfare = social_welfare[winner_index]

    # 4. Calculate distortion
    if achieved_social_welfare > 0:
        distortion = optimal_social_welfare / achieved_social_welfare
    else:
        distortion = float('inf')

    return {
        "optimal_alternative": optimal_alternative,
        "optimal_social_welfare": optimal_social_welfare,
        "chosen_alternative": winner,
        "achieved_social_welfare": achieved_social_welfare,
        "distortion": distortion
    }

#Run it

if __name__ == "__main__":
    num_agents = int(input("Please enter the number of agents here"))
    num_alternatives = int(input("Please enter the number of alternatives here"))
    distribution_type = str.lower(input("Please select type of distribution either 'uniform', 'normal', 'strong_preference', or 'unit_some': "))
    
    print("=== Social Choice Analysis: Plurality Voting ===")
    print(f"Parameters:")
    print(f"  - Number of Agents: {num_agents}")
    print(f"  - Number of Alternatives: {num_alternatives}")
    print(f"  - Distribution Type: {distribution_type}")
    print(f"  - Voting Rule: Plurality\n")
    
    # Generate the two crucial datasets
    cardinal_utilities = generate_utilities(num_agents, num_alternatives, distribution_type)
    ordinal_preferences = generate_preferences(num_agents, num_alternatives)
    
    # Run the voting algorithm to get the "Chosen Alternative"
    plurality_results = plurality_voting(ordinal_preferences)
    plurality_winner = plurality_results['winner']
    
    # Run the distortion calculation to get the final outputs
    distortion_results = calculate_distortion(cardinal_utilities, plurality_winner)
    
    # --- UI/UX Mock-up: Output Panel ---
    print("=== Analysis Results ===")
    print(f"  - Optimal Alternative: {distortion_results['optimal_alternative']}")
    print(f"  - Optimal Social Welfare: {distortion_results['optimal_social_welfare']:.2f}")
    print(f"  - Chosen Alternative (by Plurality): {distortion_results['chosen_alternative']}")
    print(f"  - Achieved Social Welfare: {distortion_results['achieved_social_welfare']:.2f}")
    print(f"  - Distortion: {distortion_results['distortion']:.2f}")

=== Social Choice Analysis: Plurality Voting ===
Parameters:
  - Number of Agents: 10
  - Number of Alternatives: 5
  - Distribution Type: strong_preference
  - Voting Rule: Plurality

=== Analysis Results ===
  - Optimal Alternative: A1
  - Optimal Social Welfare: 3.00
  - Chosen Alternative (by Plurality): A1
  - Achieved Social Welfare: 3.00
  - Distortion: 1.00


# Self-contained Python script for the Veto voting rule

This script provides a complete experimental framework for simulating the Veto voting rule and calculating its distortion.

It combines all the previously developed functions into a single, cohesive programme that follows the UI/UX mock-up. It takes a single set of inputs (number of agents, number of alternatives, and distribution type) and outputs the five key metrics for analysis.

In [16]:
# === PART 1: Data Generation Functions ===

def generate_utilities(num_agents, num_alternatives, distribution_type='uniform'):
    """
    Generates a 2D array of cardinal utility values based on a specified distribution.

    Args:
        num_agents (int): The number of agents (voters).
        num_alternatives (int): The number of alternatives.
        distribution_type (str): The type of distribution to use.
                                 'uniform': Utilities are randomly and uniformly distributed between 0 and 1.
                                 'normal': Utilities follow a normal (Gaussian) distribution.
                                 'strong_preference': One alternative has a utility of 1.00, others are 0.00.
                                 'unit_sum': All utilities for a single agent sum to 1.
    Returns:
        list of list: A nested list where utilities[i][j] is the utility of alternative j for agent i.
    """
    utilities = []
    
    # Check for valid distribution type
    valid_distributions = ['uniform', 'normal', 'strong_preference', 'unit_sum']
    if distribution_type not in valid_distributions:
        print(f"Warning: Invalid distribution type '{distribution_type}'. Defaulting to 'uniform'.")
        distribution_type = 'uniform'
        
    # Generate utilities based on the specified distribution
    for _ in range(num_agents):
        agent_utilities = []
        if distribution_type == 'uniform':
            # Uniform distribution: each utility value is equally likely to be between 0 and 1.
            agent_utilities = [random.uniform(0, 1) for _ in range(num_alternatives)]
        elif distribution_type == 'normal':
            # Normal distribution: values are clustered around a mean.
            agent_utilities = [np.random.normal(loc=0.5, scale=0.2) for _ in range(num_alternatives)]
            # We'll clip the values to ensure they stay within the [0, 1] range.
            agent_utilities = [max(0, min(1, u)) for u in agent_utilities]
        elif distribution_type == 'strong_preference':
            # Strong Preference: one alternative is a clear favorite (1.00), all others are worthless (0.00).
            agent_utilities = [0.00] * num_alternatives
            strong_choice_index = random.randint(0, num_alternatives - 1)
            agent_utilities[strong_choice_index] = 1.00
        elif distribution_type == 'unit_sum':
            # Unit-Sum: utilities are randomly distributed such that their sum is 1.
            # We generate random points and use them to partition the interval [0, 1].
            points = sorted([0.0] + [random.uniform(0, 1) for _ in range(num_alternatives - 1)] + [1.0])
            agent_utilities = [points[i+1] - points[i] for i in range(num_alternatives)]
            
        utilities.append(agent_utilities)
        
    return utilities

def generate_random_preference_list(num_alternatives):
    """
    Generates a random permutation of alternatives to represent an agent's preferences.
    """
    alternatives = [f'A{i + 1}' for i in range(num_alternatives)]
    random.shuffle(alternatives)
    return alternatives

def generate_preferences(num_agents, num_alternatives):
    """
    Simulates preferences for a given number of agents and alternatives.
    """
    preferences = []
    for _ in range(num_agents):
        preferences.append(generate_random_preference_list(num_alternatives))
    return preferences

# === PART 2: Voting Algorithm Function ===

def veto_voting(preferences):
    """
    Implements the Veto voting rule to find the winning alternative.
    Veto works by giving one negative point to the last-ranked alternative of each agent.
    The alternative with the highest total score (fewest vetoes) wins.

    Args:
        preferences (list): A 2D list of agent preferences.

    Returns:
        dict: A dictionary containing the winner, their score, and all scores.
    """
    if not preferences or not preferences[0]:
        return {"winner": None, "score": 0, "scores": {}}

    num_alternatives = len(preferences[0])
    veto_scores = {alt: 0 for alt in preferences[0]}

    for preference_list in preferences:
        last_choice = preference_list[num_alternatives - 1]
        veto_scores[last_choice] += 1
    
    winner = None
    min_vetoes = float('inf')

    for alternative, score in veto_scores.items():
        if score < min_vetoes:
            min_vetoes = score
            winner = alternative
    
    scores_for_display = {alt: len(preferences) - score for alt, score in veto_scores.items()}

    return {
        "winner": winner,
        "score": len(preferences) - min_vetoes,
        "scores": scores_for_display
    }

# === PART 3: Distortion Calculation Function ===

def calculate_distortion(utilities, winner):
    """
    Calculates the distortion of a voting rule's outcome.
    
    The distortion is the ratio of the optimal social welfare (the best possible outcome)
    to the achieved social welfare (the outcome of the chosen alternative).

    Args:
        utilities (list): A 2D list of cardinal utility values for each agent and alternative.
                          utilities[i][j] is the utility of alternative j for agent i.
        winner (str): The name of the winning alternative as determined by a voting rule
                      (e.g., 'A3').

    Returns:
        dict: A dictionary containing the five key outputs for analysis.
    """
    if not utilities:
        return {
            "optimal_alternative": None,
            "optimal_social_welfare": 0,
            "chosen_alternative": winner,
            "achieved_social_welfare": 0,
            "distortion": 0
        }

    num_agents = len(utilities)
    num_alternatives = len(utilities[0])
    
    alternative_names = [f'A{i + 1}' for i in range(num_alternatives)]
    alt_to_index = {name: i for i, name in enumerate(alternative_names)}
    
    # 1. Calculate social welfare for all alternatives
    social_welfare = [0] * num_alternatives
    for j in range(num_alternatives):
        social_welfare[j] = sum(utilities[i][j] for i in range(num_agents))
        
    # 2. Find optimal alternative and optimal social welfare
    optimal_social_welfare = max(social_welfare)
    optimal_index = social_welfare.index(optimal_social_welfare)
    optimal_alternative = alternative_names[optimal_index]
    
    # 3. Find achieved social welfare of the winning alternative
    achieved_social_welfare = 0
    if winner in alt_to_index:
        winner_index = alt_to_index[winner]
        achieved_social_welfare = social_welfare[winner_index]

    # 4. Calculate distortion
    if achieved_social_welfare > 0:
        distortion = optimal_social_welfare / achieved_social_welfare
    else:
        distortion = float('inf')

    return {
        "optimal_alternative": optimal_alternative,
        "optimal_social_welfare": optimal_social_welfare,
        "chosen_alternative": winner,
        "achieved_social_welfare": achieved_social_welfare,
        "distortion": distortion
    }

# Run it

if __name__ == "__main__":
    num_agents = int(input("Please enter the number of agents here: "))
    num_alternatives = int(input("Please enter the number of alternatives here: "))
    distribution_type = str.lower(input("Please select type of distribution either 'uniform', 'normal', 'strong_preference', or 'unit_some': "))
    
    print("\n=== Social Choice Analysis: Veto Voting ===")
    print(f"Parameters:")
    print(f"  - Number of Agents: {num_agents}")
    print(f"  - Number of Alternatives: {num_alternatives}")
    print(f"  - Distribution Type: {distribution_type}")
    print(f"  - Voting Rule: Veto\n")
    
    # Generate the two crucial datasets
    cardinal_utilities = generate_utilities(num_agents, num_alternatives, distribution_type)
    ordinal_preferences = generate_preferences(num_agents, num_alternatives)
    
    # Run the voting algorithm to get the "Chosen Alternative"
    veto_results = veto_voting(ordinal_preferences)
    veto_winner = veto_results['winner']
    
    # Run the distortion calculation to get the final outputs
    distortion_results = calculate_distortion(cardinal_utilities, veto_winner)
    
    # --- UI/UX Mock-up: Output Panel ---
    print("\n=== Analysis Results ===")
    print(f"  - Optimal Alternative: {distortion_results['optimal_alternative']}")
    print(f"  - Optimal Social Welfare: {distortion_results['optimal_social_welfare']:.2f}")
    print(f"  - Chosen Alternative (by Veto): {distortion_results['chosen_alternative']}")
    print(f"  - Achieved Social Welfare: {distortion_results['achieved_social_welfare']:.2f}")
    print(f"  - Distortion: {distortion_results['distortion']:.2f}")



=== Social Choice Analysis: Veto Voting ===
Parameters:
  - Number of Agents: 10
  - Number of Alternatives: 5
  - Distribution Type: strong_preference
  - Voting Rule: Veto


=== Analysis Results ===
  - Optimal Alternative: A4
  - Optimal Social Welfare: 4.00
  - Chosen Alternative (by Veto): A3
  - Achieved Social Welfare: 2.00
  - Distortion: 2.00


# # Self-contained Python script for the Borda voting rule

This script provides a complete experimental framework for simulating the Borda voting rule and calculating its distortion.

It combines all the previously developed functions into a single, cohesive program that follows the UI/UX mock-up. It takes a single set of inputs (number of agents, number of alternatives, and distribution type) and outputs the five key metrics for analysis.

In [17]:
# === PART 1: Data Generation Functions ===

def generate_utilities(num_agents, num_alternatives, distribution_type='uniform'):
    """
    Generates a 2D array of cardinal utility values based on a specified distribution.

    Args:
        num_agents (int): The number of agents (voters).
        num_alternatives (int): The number of alternatives.
        distribution_type (str): The type of distribution to use.
                                 'uniform': Utilities are randomly and uniformly distributed between 0 and 1.
                                 'normal': Utilities follow a normal (Gaussian) distribution.
                                 'strong_preference': One alternative has a utility of 1.00, others are 0.00.
                                 'unit_sum': All utilities for a single agent sum to 1.
    Returns:
        list of list: A nested list where utilities[i][j] is the utility of alternative j for agent i.
    """
    utilities = []
    
    # Check for valid distribution type
    valid_distributions = ['uniform', 'normal', 'strong_preference', 'unit_sum']
    if distribution_type not in valid_distributions:
        print(f"Warning: Invalid distribution type '{distribution_type}'. Defaulting to 'uniform'.")
        distribution_type = 'uniform'
        
    # Generate utilities based on the specified distribution
    for _ in range(num_agents):
        agent_utilities = []
        if distribution_type == 'uniform':
            # Uniform distribution: each utility value is equally likely to be between 0 and 1.
            agent_utilities = [random.uniform(0, 1) for _ in range(num_alternatives)]
        elif distribution_type == 'normal':
            # Normal distribution: values are clustered around a mean.
            agent_utilities = [np.random.normal(loc=0.5, scale=0.2) for _ in range(num_alternatives)]
            # We'll clip the values to ensure they stay within the [0, 1] range.
            agent_utilities = [max(0, min(1, u)) for u in agent_utilities]
        elif distribution_type == 'strong_preference':
            # Strong Preference: one alternative is a clear favorite (1.00), all others are worthless (0.00).
            agent_utilities = [0.00] * num_alternatives
            strong_choice_index = random.randint(0, num_alternatives - 1)
            agent_utilities[strong_choice_index] = 1.00
        elif distribution_type == 'unit_sum':
            # Unit-Sum: utilities are randomly distributed such that their sum is 1.
            # We generate random points and use them to partition the interval [0, 1].
            points = sorted([0.0] + [random.uniform(0, 1) for _ in range(num_alternatives - 1)] + [1.0])
            agent_utilities = [points[i+1] - points[i] for i in range(num_alternatives)]
            
        utilities.append(agent_utilities)
        
    return utilities

def generate_random_preference_list(num_alternatives):
    """
    Generates a random permutation of alternatives to represent an agent's preferences.
    """
    alternatives = [f'A{i + 1}' for i in range(num_alternatives)]
    random.shuffle(alternatives)
    return alternatives

def generate_preferences(num_agents, num_alternatives):
    """
    Simulates preferences for a given number of agents and alternatives.
    """
    preferences = []
    for _ in range(num_agents):
        preferences.append(generate_random_preference_list(num_alternatives))
    return preferences

# === PART 2: Voting Algorithm Function ===

def borda_voting(preferences):
    """
    Implements the Borda voting rule to find the winning alternative.

    Args:
        preferences (list): A 2D list of agent preferences.

    Returns:
        dict: A dictionary containing the winner, their score, and all scores.
    """
    if not preferences or not preferences[0]:
        return {"winner": None, "score": 0, "scores": {}}
    
    num_alternatives = len(preferences[0])
    borda_scores = collections.defaultdict(int)

    # Calculate points for each alternative based on its rank
    for preference_list in preferences:
        for i, alternative in enumerate(preference_list):
            points = num_alternatives - 1 - i
            borda_scores[alternative] += points
    
    # Find the winner: the alternative with the highest Borda score.
    winner = None
    max_score = -1

    for alternative, score in borda_scores.items():
        if score > max_score:
            max_score = score
            winner = alternative
    
    return {
        "winner": winner,
        "score": max_score,
        "scores": dict(borda_scores)
    }

# === PART 3: Distortion Calculation Function ===

def calculate_distortion(utilities, winner):
    """
    Calculates the distortion of a voting rule's outcome.
    
    The distortion is the ratio of the optimal social welfare (the best possible outcome)
    to the achieved social welfare (the outcome of the chosen alternative).

    Args:
        utilities (list): A 2D list of cardinal utility values for each agent and alternative.
                          utilities[i][j] is the utility of alternative j for agent i.
        winner (str): The name of the winning alternative as determined by a voting rule
                      (e.g., 'A3').

    Returns:
        dict: A dictionary containing the five key outputs for analysis.
    """
    if not utilities:
        return {
            "optimal_alternative": None,
            "optimal_social_welfare": 0,
            "chosen_alternative": winner,
            "achieved_social_welfare": 0,
            "distortion": 0
        }

    num_agents = len(utilities)
    num_alternatives = len(utilities[0])
    
    alternative_names = [f'A{i + 1}' for i in range(num_alternatives)]
    alt_to_index = {name: i for i, name in enumerate(alternative_names)}
    
    # 1. Calculate social welfare for all alternatives
    social_welfare = [0] * num_alternatives
    for j in range(num_alternatives):
        social_welfare[j] = sum(utilities[i][j] for i in range(num_agents))
        
    # 2. Find optimal alternative and optimal social welfare
    optimal_social_welfare = max(social_welfare)
    optimal_index = social_welfare.index(optimal_social_welfare)
    optimal_alternative = alternative_names[optimal_index]
    
    # 3. Find achieved social welfare of the winning alternative
    achieved_social_welfare = 0
    if winner in alt_to_index:
        winner_index = alt_to_index[winner]
        achieved_social_welfare = social_welfare[winner_index]

    # 4. Calculate distortion
    if achieved_social_welfare > 0:
        distortion = optimal_social_welfare / achieved_social_welfare
    else:
        distortion = float('inf')

    return {
        "optimal_alternative": optimal_alternative,
        "optimal_social_welfare": optimal_social_welfare,
        "chosen_alternative": winner,
        "achieved_social_welfare": achieved_social_welfare,
        "distortion": distortion
    }

# Run it

if __name__ == "__main__":
    num_agents = int(input("Please enter the number of agents here: "))
    num_alternatives = int(input("Please enter the number of alternatives here: "))
    distribution_type = str.lower(input("Please select type of distribution either 'uniform', 'normal', 'strong_preference', or 'unit_some': "))
    
    print("\n=== Social Choice Analysis: Borda Voting ===")
    print(f"Parameters:")
    print(f"  - Number of Agents: {num_agents}")
    print(f"  - Number of Alternatives: {num_alternatives}")
    print(f"  - Distribution Type: {distribution_type}")
    print(f"  - Voting Rule: Borda\n")
    
    # Generate the two crucial datasets
    cardinal_utilities = generate_utilities(num_agents, num_alternatives, distribution_type)
    ordinal_preferences = generate_preferences(num_agents, num_alternatives)
    
    # Run the voting algorithm to get the "Chosen Alternative"
    borda_results = borda_voting(ordinal_preferences)
    borda_winner = borda_results['winner']
    
    # Run the distortion calculation to get the final outputs
    distortion_results = calculate_distortion(cardinal_utilities, borda_winner)
    
    # --- UI/UX Mock-up: Output Panel ---
    print("\n=== Analysis Results ===")
    print(f"  - Optimal Alternative: {distortion_results['optimal_alternative']}")
    print(f"  - Optimal Social Welfare: {distortion_results['optimal_social_welfare']:.2f}")
    print(f"  - Chosen Alternative (by Borda): {distortion_results['chosen_alternative']}")
    print(f"  - Achieved Social Welfare: {distortion_results['achieved_social_welfare']:.2f}")
    print(f"  - Distortion: {distortion_results['distortion']:.2f}")



=== Social Choice Analysis: Borda Voting ===
Parameters:
  - Number of Agents: 10
  - Number of Alternatives: 5
  - Distribution Type: strong_preference
  - Voting Rule: Borda


=== Analysis Results ===
  - Optimal Alternative: A4
  - Optimal Social Welfare: 4.00
  - Chosen Alternative (by Borda): A3
  - Achieved Social Welfare: 0.00
  - Distortion: inf
